In [1]:
#

In [2]:
import pandas as pd
import regex as re
import os
from pathlib import Path
import json


In [3]:
from extract_persona_b_output import main

In [4]:

df = pd.read_pickle('./RESULTS.pkl')
df['RESULT'] = df['RESULT'].apply(lambda x: str(x).strip().removeprefix("```xml").removesuffix("```").strip())

df['RESULT'].apply(lambda x: (x[:10], x[-10:])).value_counts()


RESULT
(<?xml vers, \n</module>)    48
Name: count, dtype: int64

In [5]:

# Process XML strings from DataFrame column
df['RESULT'].apply(lambda x: main(
    input_file=x, 
    path=False,  # Key parameter!
    output_dir='./EXPORT',
    save_plan=False,
    save_snippets=False
))

📖 Processing XML string

📦 EXTRACTION SUMMARY
Module Name: Mean Girls: Context API
Slug: mean-girls-context-api

Files extracted: 1
  - src/modules/mean-girls-context-api/index.tsx

Integration snippets: 3
  - route
  - import
  - home_card

✅ Written: EXPORT\modules\mean-girls-context-api\index.tsx

✨ Extraction complete!

📋 Next steps:
1. Add import to App.tsx:
   import MeanGirlsContextAPI from "@modules/mean-girls-context-api";

2. Add route to App.tsx:
   <Route
  path="/mean-girls-context-api"
  element={
    <ModuleWrapper
      bgClass="bg-white"
      textClass="text-slate-700"
      fontClass="font-sans"
    >
      <MeanGirlsContextAPI />
    </ModuleWrapper>
  }
/>

3. Add card to src/modules/home/index.tsx:
   {
  path: "/mean-girls-context-api",
  title: "Mean Girls",
  subtitle: "North Shore High, 2004",
  concept: "Context API & Prop Drilling",
  icon: Sparkles,
  colorClass: "text-pink-500",
  bgClass: "bg-pink-50 border-pink-300 hover:border-pink-500"
}
📖 Processing X

0     {'success': True, 'files_written': 1, 'output_...
1     {'success': True, 'files_written': 1, 'output_...
2     {'success': True, 'files_written': 1, 'output_...
3     {'success': True, 'files_written': 1, 'output_...
4     {'success': True, 'files_written': 1, 'output_...
5     {'success': True, 'files_written': 1, 'output_...
6     {'success': True, 'files_written': 1, 'output_...
7     {'success': True, 'files_written': 1, 'output_...
8     {'success': True, 'files_written': 1, 'output_...
9     {'success': True, 'files_written': 1, 'output_...
10    {'success': True, 'files_written': 1, 'output_...
11    {'success': True, 'files_written': 1, 'output_...
12    {'success': True, 'files_written': 1, 'output_...
13    {'success': True, 'files_written': 1, 'output_...
14    {'success': True, 'files_written': 1, 'output_...
15    {'success': True, 'files_written': 1, 'output_...
16    {'success': True, 'files_written': 1, 'output_...
17    {'success': True, 'files_written': 1, 'out